# Thresholding

## Simple thresholding

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import skimage.color
import skimage.filters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import skimage.color
import skimage.filters

# load the image
image = skimage.io.imread("../../fig/06-junk-before.jpg")

# convert the image to grayscale
gray_image = skimage.color.rgb2gray(image)

In [ ]:
# create a histogram of the blurred grayscale image
histogram, bin_edges = np.histogram(gray_image, bins=256, range=(0.0, 1.0))

plt.plot(bin_edges[0:-1], histogram)
plt.title("Grayscale Histogram")
plt.xlabel("grayscale value")
plt.ylabel("pixels")
plt.xlim(0, 1.0)
plt.show()

In [ ]:
# create a mask based on the threshold
t = 0.8
binary_mask = gray_image < t
skimage.io.imshow(binary_mask)
plt.show()

In [ ]:
# use the binary_mask to select the "interesting" part of the image
selection = np.zeros_like(image)
selection[binary_mask] = image[binary_mask]

skimage.io.imshow(selection)
plt.show()

### More practice with simple thresholding

In [ ]:
image = skimage.io.imread("../../fig/06-more-junk.jpg")
gray_image = skimage.color.rgb2gray(image)
histogram, bin_edges = np.histogram(gray_image, bins=256, range=(0.0, 1.0))

plt.plot(bin_edges[0:-1], histogram)
plt.title("Graylevel histogram")
plt.xlabel("gray value")
plt.ylabel("pixel count")
plt.xlim(0, 1.0)
plt.show()

In [ ]:
t = 0.5
binary_mask = gray_image > t

skimage.io.imshow(binary_mask)
plt.show()

In [ ]:
selection = np.zeros_like(image)
selection[binary_mask] = image[binary_mask]

skimage.io.imshow(selection)
plt.show()

## Adaptive thresholding